In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from sklearn.preprocessing import LabelEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

Using TensorFlow backend.


In [2]:
balanced_data=pd.read_csv('clothing.csv')
balanced_data=balanced_data[['Category','Description']]
data=balanced_data

In [3]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
#Found 400000 word vectors.


Found 400000 word vectors.


In [4]:

values = array(data['Category'])
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
labels=onehot_encoded

[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.

In [5]:
docs=np.array(data['Description'])
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[47, 9, 2, 50, 51, 194, 8, 6, 548, 62, 70, 37, 2, 11, 38, 14, 129, 31, 549, 31, 550, 7, 253, 1, 551, 7, 1, 387, 50, 51, 194, 39, 388, 2, 42, 9, 130, 5, 6, 62, 131, 16, 4, 63, 389, 112, 195, 3, 390, 26, 50, 51, 194, 18, 32, 19, 48, 196, 552, 197, 553, 12], [27, 2, 198, 158, 28, 159, 1, 199, 391, 8, 6, 554, 62, 21, 555, 6, 132, 71, 254, 62, 16, 11, 72, 3, 198, 158, 17, 9, 1, 22, 31, 255, 31, 160, 112, 113, 392, 1, 256, 556, 23, 73, 57, 74, 22, 133, 4, 557, 257, 3, 58, 113, 32, 19, 33, 10, 95, 52, 10, 47, 9, 1, 198, 158, 7, 161], [4, 558, 559, 560, 65, 561, 53, 8, 4, 562, 563, 112, 5, 4, 393, 195, 564, 258, 3, 200, 24, 1, 75, 565, 566, 5, 54, 40, 567, 568, 3, 394, 569, 570, 571, 201, 113, 395], [572, 96, 4, 199, 259, 33, 10, 21, 260, 2, 162, 114, 8, 573, 574, 575, 576, 577, 578, 254, 261, 579, 19, 33, 10, 132, 3, 262, 2, 202, 203, 38, 6, 580, 204, 58, 19, 33, 1, 581, 582, 52, 29, 26, 583, 17, 3, 160, 11, 205, 134, 584, 2, 42, 135, 59, 86, 7, 47, 1, 162, 114, 7, 161], [1, 263, 57, 396, 12

In [6]:
# get the longest document 
max_length=0
for each in encoded_docs:
    if len(each)>=max_length:
        max_length=len(each)
max_length
    

130

In [7]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 47   9   2 ...,   0   0   0]
 [ 27   2 198 ...,   0   0   0]
 [  4 558 559 ...,   0   0   0]
 ..., 
 [307 150 234 ...,   0   0   0]
 [338 129 542 ...,   0   0   0]
 [201 183 143 ...,   0   0   0]]


In [8]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)
x_train.shape
x_train

array([[  60,   77,    2, ...,    0,    0,    0],
       [ 979,  980,  981, ...,    0,    0,    0],
       [  82,    2,   30, ...,    0,    0,    0],
       ..., 
       [1007,   26, 1008, ...,    0,    0,    0],
       [ 660,  417,  418, ...,    0,    0,    0],
       [   2,  346,   36, ...,    0,    0,    0]], dtype=int32)

In [ ]:
data_dim = 100
timesteps = 130
num_classes = 9

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=130, trainable=False)
model.add(e)

model.add(LSTM(9, return_sequences=False,
              input_shape=(timesteps, data_dim),activation='softmax'))
#model.add(LSTM(9, return_sequences=True,
#               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 9
#model.add(LSTM(9))  # return a single vector of dimension 32
#model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])


print(model.summary())
model.fit(x_train, y_train,
          batch_size=1,
          epochs=100)
# evaluate the model
loss, accuracy = model.evaluate(x_val, y_val, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 100)          114000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 9)                 3960      
Total params: 117,960
Trainable params: 3,960
Non-trainable params: 114,000
_________________________________________________________________
None
Epoch 1/100
72/72 [==============================] - 38s - loss: 2.2039 - acc: 0.0694          